In [1]:
import sys
import pandas as pd 
import numpy as np 
sys.path.append('../src/')
import load_transform_data

transformed = load_transform_data.TransformXY()
transformed.transform_data('LabelEncoder')

../src/load_transform_data.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_y['final_teo'] = test_y['final_teo'].apply(lambda x: self.le_dict.get(x, -1))


In [2]:
from sklearn.ensemble import RandomForestClassifier

X_train = np.concatenate([transformed.X_train_tone,transformed.X_train_initial,\
                          transformed.X_train_final], axis=1)
X_train.shape

(640, 64)

In [3]:
X_test = np.concatenate([transformed.X_test_tone,transformed.X_test_initial,\
                          transformed.X_test_final], axis=1)

In [4]:
#from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier

RFs = [RandomForestClassifier(n_estimators=100, max_samples=0.7, random_state=i) for i in range(3)]

Y_train = [transformed.Y_train_tone,transformed.Y_train_initial,transformed.Y_train_final]

for idx, rf in enumerate(RFs):
    rf.fit(X_train, Y_train[idx])

pred_tone = np.array(RFs[0].predict(X_test))
pred_initial = np.array(RFs[1].predict(X_test))
pred_final = np.array(RFs[2].predict(X_test))

In [5]:
transformed.Y_test_final

697    53
668    15
63     10
534    14
66      3
       ..
799    15
745    62
513    48
671    41
264    -1
Name: final_teo, Length: 161, dtype: int64

In [6]:
feature_imp0 = RFs[0].feature_importances_.reshape(-1, 1)
feature_imp1 = RFs[1].feature_importances_.reshape(-1, 1)
feature_imp2 = RFs[2].feature_importances_.reshape(-1, 1)

In [16]:
cat = transformed.encX.categories_
man_feature = np.concatenate([cat[0], cat[1], cat[2]], axis=0).reshape(-1, 1)
feature_initial = pd.DataFrame(np.concatenate([feature_imp0, man_feature], axis=1), \
                               columns=['feature_importance', 'features'])
feature_initial.sort_values(by='feature_importance', ascending=False).head(10)                         

,feature_importance,features
0,0.1418,1
1,0.104914,2
2,0.0973116,3
3,0.0763801,4
38,0.0281555,i
5,0.0227322,b
12,0.0211497,j
27,0.0195108,zh
14,0.0191988,l
21,0.0179787,sh


In [8]:
pred_tone.shape, pred_initial.shape, pred_final.shape

((161,), (161,), (161,))

In [9]:
pred_tone_df = transformed.encT.inverse_transform(pred_tone)
pred_initial_df = transformed.encI.inverse_transform(pred_initial)
pred_final_df = transformed.encF.inverse_transform(pred_final)

pred_df = np.column_stack((pred_tone_df.T, pred_initial_df.T, pred_final_df.T))
#len(pred_tone_df), len(pred_initial_df), len(pred_final_df), pred_tone.shape
transformed.encF.classes_

array(['a', 'ai', 'aim', 'ak', 'am', 'an', 'ap', 'au', 'aŋ', 'aŋ̍', 'aʔ',
       'e', 'ek', 'eŋ', 'eʔ', 'i', 'ia', 'iak', 'iaŋ', 'iaʔ', 'ie', 'iem',
       'ieŋ', 'ieʔ', 'ik', 'im', 'ip', 'iu', 'iŋ', 'iəm', 'iəp', 'iəu',
       'iʔ', 'iẽ∼', 'o', 'oi', 'oiʔ', 'ok', 'ou', 'oŋ', 'oʔ', 'u', 'ua',
       'uai', 'uaŋ', 'uaʔ', 'ue', 'ueŋ', 'ueʔ', 'ui', 'uk', 'un', 'uŋ',
       'ã∼', 'õĩ∼', 'õũ∼', 'ĩã∼', 'ĩũ∼', 'ĩẽ∼', 'ĩ∼', 'ŋ', 'ŋ̍', 'ũã∼',
       'ũĩ∼', 'ũẽ∼', 'ɯ', 'ɯk', 'ɯŋ', 'ɿ', 'ẽ∼'], dtype='<U3')

In [10]:
test_tone_df = transformed.encT.inverse_transform(transformed.Y_test_tone)
test_initial_df = transformed.encI.inverse_transform(transformed.Y_test_initial)
outliers = ['-1', '53']
test_final_df = transformed.Y_test_final.apply(lambda x: \
                                               transformed.encF.inverse_transform(np.array([x]))[0]
                                               if str(x) not in outliers else 'unknown')
#test_final_df = transformed.Y_test_final.apply(lambda x: x if x not in outliers else 'unknown')

In [11]:
test_final_df[test_final_df=='unknown']

697    unknown
360    unknown
667    unknown
481    unknown
231    unknown
49     unknown
365    unknown
76     unknown
765    unknown
425    unknown
541    unknown
264    unknown
Name: final_teo, dtype: object

In [12]:
test_df = np.column_stack((test_tone_df.T, test_initial_df.T, test_final_df.T))

In [13]:
#test_df

In [14]:
results = np.sum(pred_df==test_df, axis=0)
print(results[0], results[1], results[2])

85 103 52


In [15]:
# Accuracy for teochew_tones, teochew_initials, and teochew_finals 
print('accuracy for tones prediction: {}%'.format(results[0]/pred_tone.shape[0]*100))
print('accuracy for initials prediction: {}%'.format(results[1]/pred_initial.shape[0]*100))
print('accuracy for finals prediction: {}%'.format(results[2]/pred_final.shape[0]*100))

cols = [['citation', 'initial', 'final', 'pred_citation', 'pred_initial', 'pred_final']]
result_df = pd.DataFrame(np.concatenate([test_df, pred_df], axis=1), columns=cols)

result_df.columns = ['citation','initial','final',
                     'pred_citation','pred_initial','pred_final']
result_df['pred_final'].isnull().sum()

result_df[(result_df['citation']==result_df['pred_citation'])
          &(result_df['initial']==result_df['pred_initial'])
          &(result_df['final']==result_df['pred_final'])]

# harsh metrics, when all three predictions are equal 
26/pred_tone.shape[0]*100

result_df.to_csv('../output/rf.csv', index = None, header=True)

accuracy for tones prediction: 52.79503105590062%
accuracy for initials prediction: 63.975155279503106%
accuracy for finals prediction: 32.298136645962735%
